In [1]:
import os

import pywikibot as pwb
# import pywikibot.pywikibot as pwb # IMPORTING pywikibot
from PDF_module.pdftotext import convert_pdf_to_text # IMPORTING MY PYTHON SCRIPT WHICH ADDS A TEXT LAYER

In [4]:
pwb.login

<module 'pywikibot.login' from '/home/happy/.local/lib/python3.10/site-packages/pywikibot/login.py'>

In [4]:
# def user_login(_wiki_url, _username, _password):
#     _site = pywikibot.Site(site = _wiki_url, use)
#     _site.login(user=_username, password=_password)
#     return _site
# from urllib.parse import urlparse

# def user_login(wiki_url, username, password):
def user_login():
    # url_parts = urlparse(wiki_url)
    # print(url_parts)
    # input()
    # family_name = 'localhost'
    # site = pywikibot.Site(url=url_parts.scheme + '://' + url_parts.netloc, fam=family_name)
    # site = pwb.Site(url = wiki_url, fam=family_name)
    # pywikibot.config.password_file = password
    _site = pwb.login()
    return _site

def user_logout(_site):
    _site.logout()


In [6]:
def upload_pdf_files(file_path, wiki_site):
    if not wiki_site.logged_in():
        raise Exception("User is not logged in.")
    
    for file_name in os.listdir(file_path):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(file_path, file_name)
            page_name = file_name[:-4]  # Remove the '.pdf' extension from the file name

            page = pwb.FilePage(wiki_site, page_name)
            page.upload(file_path, ignore_warnings=True)



In [14]:
# Provide the necessary information here
file_path = './PDFs/'  # Path to the folder containing PDF files
wiki_url = 'http://localhost/'  # URL of your local MediaWiki server
username = 'mediawiki_admin'  # Your MediaWiki username
password = 'mediawiki@123'  # Your MediaWiki password


In [5]:
# site = user_login(wiki_url, username, password)
site = user_login()

TypeError: 'module' object is not callable

In [79]:
upload_pdf_files(file_path, site)

NameError: name 'site' is not defined

In [8]:
upload_pdf_files()

TypeError: upload_pdf_files() missing 2 required positional arguments: 'file_path' and 'wiki_site'

In [5]:
site = pwb.Site(code="en", fam="mediawiki")

In [4]:
site

APISite("en", "mediawiki")

In [15]:
site.login()

In [13]:
site.logout()

In [27]:
site.upload(filepage="./PDFs",)

AttributeError: 'str' object has no attribute 'text'

In [1]:
from IPython.display import clear_output

In [2]:
import os
from pathlib import Path
import pywikibot
from PDF_module.pdftotext import convert_pdf_to_text, extract_text # IMPORTING MY PYTHON SCRIPT WHICH ADDS A TEXT LAYER



def upload_pdf_files(file_path, wiki_url, username, password):
    site = pywikibot.Site(user=username, code="en", fam="mediawiki")
    site.login()

    for file_name in os.listdir(file_path):
        if file_name.endswith('.pdf'):
            print("processing ", {file_name})
            _file_path = file_path
            ffile_path = os.path.join(file_path, file_name)

            output_file = "new/ocr_"+file_name
            output_file_path = os.path.join(_file_path, output_file)
            
            convert_pdf_to_text(pdf_file=ffile_path, output_file=output_file_path)

            clear_output(wait = True)

            print("Text Layer Added")

            page_name = output_file[4:-4]  # Remove the '.pdf' extension from the file name

            # summary = "Uploaded PDF file: {}".format(page_name)  # Sample summary
            textt = extract_text(output_file_path)
            summary = "\n".join([textt[page] for page in range(len(textt))])
            
            print(summary)

            page = pywikibot.FilePage(site, page_name)
            
            try:
                page.upload(output_file_path, ignore_warnings=True, comment=summary)
            except Exception as e:
                print(e)


# Provide the necessary information here
file_path = './PDFs'  # Path to the folder containing PDF files
wiki_url = 'http://localhost'  # URL of your local MediaWiki server
username = 'mediawiki_admin'  # Your MediaWiki username
password = 'mediawiki@123'  # Your MediaWiki password

upload_pdf_files(file_path, wiki_url, username, password)


Sleeping for 19.5 seconds, 2023-07-05 20:55:01


Text Layer Added
01/07/23 pdf1.txt 1
ﬁle:///home/happy/Desktop/pdf1.txttesting	pdf	file	upload	from	MWbot


Upload successful.


In [3]:
site.logout()

NameError: name 'site' is not defined